# 1.0 Get Zoopla Data
Use the Zoopla API to pull current house listings

In [1]:
from datetime import date
import os
import math
import pandas as pd
pd.set_option('display.max_columns', 100)
from zoopla import Zoopla

/home/robertv/anaconda3/envs/datascience2/lib/python3.11/site-packages/marshmallow/schema.py:200: ChangedInMarshmallow3Warning: The dateformat option is renamed to datetimeformat in marshmallow 3.
  warnings.warn(


In [2]:
# Call an instance of Zoopla with the secret API key
zoopla_instance = Zoopla(api_key=os.environ['ZOOPLA_API_KEY'])

### Download property data for Nuneaton or Hinckley, including recently sold properties

First get total number of properties - we can only access a maximum of 100 results per call, so will need to call the API multiple times

In [3]:
AREA = 'Nuneaton'
SAVE_FOLDER = os.path.join('data', 'raw')
PAGE_SIZE = 100

In [4]:
properties = zoopla_instance.property_listings({
    'area': AREA,
    'order_by': 'age',
    'listing_status': 'sale',
    'include_sold': '1',
    'summarised': 'yes',
    'page_size': PAGE_SIZE
})

number_properties = properties['result_count']

print(f'Number of {AREA} properties = {number_properties}')

Number of Nuneaton properties = 728


Now get all properties for the area and concatenate into a dataframe

In [5]:
number_pages = math.ceil(number_properties/PAGE_SIZE)

for i in range(number_pages):
    properties = zoopla_instance.property_listings({
        'area': AREA,
        'order_by': 'age',
        'listing_status': 'sale',
        'include_sold': '1',
        'summarised': 'yes',
        'page_size': PAGE_SIZE,
        'page_number': i+1
    })
    
    zoopla_df_tmp = pd.DataFrame.from_records(properties.listing)
    
    # loop over pages and concatenate into a pandas dataframe
    if i == 0:
        zoopla_df = zoopla_df_tmp
    else:
        zoopla_df = pd.concat([zoopla_df, zoopla_df_tmp], axis=0)

### Quick check of the data

In [6]:
zoopla_df.head()

,category,agent_logo,country_code,property_type,num_bedrooms,num_bathrooms,agent_phone,county,street_name,post_town,listing_status,outcode,agent_address,details_url,num_recepts,listing_id,displayable_address,image_url,first_published_date,price_modifier,agent_name,longitude,price,description,floor_plan,country,latitude,last_published_date
0,Residential,https://st.zoocdn.com/zoopla_static_agent_logo...,gb,Terraced house,2,2,024 7662 0227,Warwickshire,Tomkinson Road,Nuneaton,sale,CV10,"Bond Gate Chambers, Bond Gate, Nuneaton",https://www.zoopla.co.uk/for-sale/details/6446...,2,64462156,"Tomkinson Road, Nuneaton CV10",https://lid.zoocdn.com/354/255/260b0fbca9f9ea4...,2023-04-22 08:33:46,None,Pointons Estate Agents,-1.487903,165000.0,***buy to let investors only tenants in situ ...,None,England,52.521416,2023-04-22 08:50:53
1,Residential,https://st.zoocdn.com/zoopla_static_agent_logo...,gb,Detached house,4,3,024 7513 1114,Warwickshire,277 The Long Shoot,Nuneaton,sale,CV11,"1a - 4a Bond Gate Chambers, Nuneaton",https://www.zoopla.co.uk/for-sale/details/6445...,4,64459560,"The Long Shoot, Nuneaton CV11",https://lid.zoocdn.com/354/255/abfc27ca46df8d6...,2023-04-21 17:31:08,None,Tom Bates Estate Agents,-1.421078,525000.0,""" incedible detached home with cinema room "" W...",None,England,52.535797,2023-04-21 20:28:04
2,Residential,https://st.zoocdn.com/zoopla_static_agent_logo...,gb,Detached bungalow,3,1,024 7662 0227,Warwickshire,Crossing Gates,Nuneaton,sale,CV11,"Bond Gate Chambers, Bond Gate, Nuneaton",https://www.zoopla.co.uk/for-sale/details/6445...,1,64458751,"Crossing Gates, Nuneaton CV11",https://lid.zoocdn.com/354/255/c9d88b69e108d57...,2023-04-21 16:27:48,None,Pointons Estate Agents,-1.454462,340000.0,Pointons are delighted to offer for sale show...,None,England,52.525795,2023-04-21 16:39:38
3,Residential,https://st.zoocdn.com/zoopla_static_agent_logo...,gb,Detached house,4,2,024 7511 9743,Warwickshire,Cliveden Walk,Nuneaton,sale,CV11,"4 Hearsall Lane, Earlsdon, Coventry",https://www.zoopla.co.uk/for-sale/details/6445...,1,64456960,"Cliveden Walk, Maple Park, Nuneaton CV11",https://lid.zoocdn.com/354/255/2f9a207a5b6ece8...,2023-04-21 14:35:46,None,Maison Estates Ltd,-1.450420,420000.0,***grande & imposing 4 bedroom detached house...,None,England,52.502000,2023-04-21 14:35:46
4,Residential,https://st.zoocdn.com/zoopla_static_agent_logo...,gb,Detached house,3,0,0333 016 5458,Warwickshire,Moor Road,Nuneaton,sale,CV10,"St George's House, 56 Peter Street, Manchester",https://www.zoopla.co.uk/for-sale/details/6445...,0,64451326,"Moor Road, Nuneaton CV10",https://lid.zoocdn.com/354/255/e44f87a4df01d35...,2023-04-21 04:24:39,None,Express Estate Agency,-1.524742,300000.0,We are proud to present this Three Bedroom Det...,None,England,52.540840,2023-04-21 04:24:39


In [7]:
zoopla_df.shape

(728, 28)

### Save to csv file, appending current date

In [8]:
today = date.today()

date_stamp = today.strftime("%d%m%Y")

In [9]:
try:
    os.mkdir(SAVE_FOLDER)
except OSError:
    pass

save_file = os.path.join(SAVE_FOLDER, f'zoopla_properties_{AREA.lower()}_{date_stamp}.csv')

zoopla_df.to_csv(save_file, index=False)